# Cleaning Wine dataset

### Importing libraries and constants

In [28]:
import pandas as pd
import os
from src.constants_and_functions.constants import data_processed_folder

### Loading data

In [29]:
wine_statistics = pd.read_csv('C:\Datasets and Projects\WINE QUALITY\Wine\Data\Raw\Wine statistics (raw data).csv')
wine_statistics.head()

,Name,Rating,Color,Intensity,Aroma,Sweetness,Acidity,Alcohol,Tannin,Balance,Finish,Aftertaste
0,EQUINOX | Luchineasa | 2020,90.7,9.,9.,94,90,8.9,8.9,8.9,8.9,8.9,838
1,BAHU WINERY | Sauvignon Blanc | 2020,89.8,94,94,93,8.9,83,82,82,82,82,82
2,CRAMA BAUER | Feteasca Alba | 2019,91.9,9.6,9.6,9.5,9.1,9.1,9.1,9.0,9.0,9.0,8.9
3,BOUCHIÉ-CHATELLIER | Argile à Silex | Pouilly-...,92.8,97,9.6,9.5,92,92,92,92,9.1,9.1,9.0
4,RASOVA | Nud | 2020,92.0,9.5,9.5,944,92,92,9.1,9.1,9.0,9.0,90


## Data Exploration

In [30]:
print(wine_statistics.shape)

(876, 12)


In [31]:
print(wine_statistics.isna().sum())

Name          0
Rating        0
Color         0
Intensity     0
Aroma         0
Sweetness     0
Acidity       0
Alcohol       0
Tannin        0
Balance       0
Finish        0
Aftertaste    0
dtype: int64


In [32]:
print(wine_statistics.dtypes)

Name           object
Rating        float64
Color          object
Intensity      object
Aroma          object
Sweetness      object
Acidity        object
Alcohol        object
Tannin         object
Balance        object
Finish         object
Aftertaste     object
dtype: object


In [33]:
# Finding and deleting duplicates
duplicates = wine_statistics.duplicated().sum()
print(duplicates)

1


In [34]:
wine_statistics.drop_duplicates(keep=False,inplace=True)

### ## Splitting and changing 'Name' column, creating 'Year' column

In [35]:
wine_statistics['Name'] = wine_statistics['Name'].str.replace(r"([ ][|][ ])", ', ', regex=True)
wine_statistics['Year'] = wine_statistics.Name.str.extract(r'([0-9]{4})', expand = True)
wine_statistics['Name'] = wine_statistics['Name'].str.replace(r'([,][ ])([0-9]{4}|#)', '',  regex=True)

In [36]:
print(wine_statistics['Year'].isnull().sum())

42


In [37]:
# replacing na values
wine_statistics['Year'].fillna( method ='ffill', inplace = True)

### Numbers standardization

In [39]:
# Сharacter replacement
columns = ['Color', 'Intensity', 'Aroma', 'Sweetness', 'Acidity', 'Alcohol', 'Tannin', 'Balance',
           'Finish', 'Aftertaste']
wine_statistics[columns] = wine_statistics[columns].replace(r',', '', regex=True)
wine_statistics[columns] = wine_statistics[columns].replace(r'[.]', '', regex=True)

In [40]:
wine_statistics[columns] = wine_statistics[columns].apply(pd.to_numeric, errors='coerce')

In [41]:
print(wine_statistics.isnull().sum())

Name          0
Rating        0
Color         4
Intensity     3
Aroma         1
Sweetness     0
Acidity       0
Alcohol       0
Tannin        0
Balance       0
Finish        0
Aftertaste    0
Year          0
dtype: int64


In [42]:
# Filling empty cells
wine_statistics['Color'].fillna( method ='ffill', inplace=True)
wine_statistics['Intensity'].fillna( method ='ffill', inplace=True)
wine_statistics['Aroma'].fillna( method ='ffill', inplace=True)

In [43]:
wine_statistics.head()

,Name,Rating,Color,Intensity,Aroma,Sweetness,Acidity,Alcohol,Tannin,Balance,Finish,Aftertaste,Year
0,"EQUINOX, Luchineasa",90.7,9.0,9.0,94.0,90,89,89,89,89,89,838,2020
1,"BAHU WINERY, Sauvignon Blanc",89.8,94.0,94.0,93.0,89,83,82,82,82,82,82,2020
2,"CRAMA BAUER, Feteasca Alba",91.9,96.0,96.0,95.0,91,91,91,90,90,90,89,2019
3,"BOUCHIÉ-CHATELLIER, Argile à Silex, Pouilly-Fumé",92.8,97.0,96.0,95.0,92,92,92,92,91,91,90,2019
4,"RASOVA, Nud",92.0,95.0,95.0,944.0,92,92,91,91,90,90,90,2020


In [44]:
# Detecting too low rating values, which have been scraped not correctly
for i in wine_statistics['Rating']:
    if i<7:
        print(i)

0.9
0.7


In [45]:
# Deleting too low rating values
wine_statistics.drop(wine_statistics.index[wine_statistics['Rating'] < 10], inplace = True)

In [46]:
# Adding "Verdict" column
def label_race(row):
    if row['Rating'] >= 93:
        return 'great'
    if row['Rating'] >= 91 and row['Rating']< 93:
        return 'fine'
    if row['Rating'] >= 89 and row['Rating']< 91:
        return 'nice'
    return 'fair'     

In [47]:
wine_statistics.apply(lambda row: label_race(row), axis=1)
wine_statistics['Verdict'] = wine_statistics.apply(lambda row: label_race(row), axis=1)

### Converting wine characteristics values to the same form

In [48]:
def data_formatting(num):
    while num > 10 :
        num *= 0.1
    return round(num, 1)
for column in columns:
    wine_statistics[column] = wine_statistics[column].map(lambda x: data_formatting(x))

In [49]:
columns_names = ['Name', 'Year', 'Rating', 'Verdict', 'Color', 'Intensity', 'Aroma', 'Sweetness', 'Acidity', 'Alcohol',
                 'Tannin', 'Balance', 'Finish', 'Aftertaste' ]
wine_statistics = wine_statistics.reindex(columns=columns_names)

In [50]:
wine_statistics.head()

,Name,Year,Rating,Verdict,Color,Intensity,Aroma,Sweetness,Acidity,Alcohol,Tannin,Balance,Finish,Aftertaste
0,"EQUINOX, Luchineasa",2020,90.7,nice,9.0,9.0,9.4,9.0,8.9,8.9,8.9,8.9,8.9,8.4
1,"BAHU WINERY, Sauvignon Blanc",2020,89.8,nice,9.4,9.4,9.3,8.9,8.3,8.2,8.2,8.2,8.2,8.2
2,"CRAMA BAUER, Feteasca Alba",2019,91.9,fine,9.6,9.6,9.5,9.1,9.1,9.1,9.0,9.0,9.0,8.9
3,"BOUCHIÉ-CHATELLIER, Argile à Silex, Pouilly-Fumé",2019,92.8,fine,9.7,9.6,9.5,9.2,9.2,9.2,9.2,9.1,9.1,9.0
4,"RASOVA, Nud",2020,92.0,fine,9.5,9.5,9.4,9.2,9.2,9.1,9.1,9.0,9.0,9.0


In [52]:
# Saving dataset
os.chdir(data_processed_folder)
wine_statistics.to_csv('Wine statistics(processed data).csv', index=False)